<a href="https://colab.research.google.com/github/mehmetbozdemir24/Tubitak_1505_Proje/blob/chunking/chunk1505.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*code*

In [ ]:
import os
import sys
import pickle
import subprocess

# Gerekli paketlerin kontrolü ve kurulumu
def install_packages():
    print("Gerekli kütüphaneler kuruluyor... Lütfen bekleyin.")
    packages = [
        "langchain",
        "langchain-community",
        "pypdf",
        "openpyxl",
        "pandas",
        "python-docx",
        "python-pptx",
        "unstructured",
        "networkx"
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)
    print("Kurulum tamamlandı.")

try:
    import langchain
    import pandas as pd
except ImportError:
    install_packages()
    import pandas as pd

In [ ]:
from langchain_community.document_loaders import (
    PyPDFLoader,
    Docx2txtLoader,
    UnstructuredPowerPointLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document


In [ ]:
# Ana Dizin Yolu
BASE_PATH = "/content/drive/MyDrive/1505 Chunking/Doküman"

# Hedef Klasörler
TARGET_FOLDERS = ["pdf", "excel", "word", "powerpoint"]

def get_file_permission(filename, folder_name):

    filename_lower = filename.lower()

    if any(keyword in filename_lower for keyword in ["gizli", "maliyet", "yonetim", "admin"]):
        return "admin"

    # Örnek Kural 2: Varsayılan olarak herkes (user)
    return "user"

In [ ]:
#doküman işleme

class DriveDocumentProcessor:
    def __init__(self, chunk_size=1000, chunk_overlap=100):

        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap
        )
        self.processed_chunks = []

    def process_excel(self, file_path, permission):
        """
        Excel dosyalarını satır satır okur ve bağlamsal chunklar oluşturur.
        Her satır bir chunk veya birleştirilmiş metin olur.
        """
        try:

            xls = pd.read_excel(file_path, sheet_name=None)

            for sheet_name, df in xls.items():
                # Boş sütunları ve satırları temizle
                df.dropna(how='all', axis=0, inplace=True)
                df.dropna(how='all', axis=1, inplace=True)

                # Tarih formatlarını string'e çevir
                for col in df.columns:
                    if pd.api.types.is_datetime64_any_dtype(df[col]):
                        df[col] = df[col].dt.strftime('%Y-%m-%d')

                # Her bir satırı bir metin bloğuna dönüştür
                for index, row in df.iterrows():
                    # Satır verisini "Sütun Adı: Değer" formatına getir
                    row_content_list = []
                    for col in df.columns:
                        val = row[col]
                        if pd.notna(val) and str(val).strip() != "":
                            row_content_list.append(f"{col}: {val}")

                    if not row_content_list:
                        continue

                    # Satırı tek bir metin haline getir
                    row_text = f"Tablo: {sheet_name}, Satır: {index+1}\n" + "\n".join(row_content_list)

                    # Metadata oluştur
                    metadata = {
                        "source": os.path.basename(file_path),
                        "sheet_name": sheet_name,
                        "row_number": index + 1,
                        "chunk_number": len(self.processed_chunks) + 1,
                        "permission": permission,
                        "file_type": "excel",
                        "original_source": file_path
                    }

                    doc = Document(page_content=row_text, metadata=metadata)
                    self.processed_chunks.append(doc)

            print(f"  -> Excel işlendi: {len(xls)} sayfa.")

        except Exception as e:
            print(f"  !! Excel hatası ({os.path.basename(file_path)}): {e}")

    def process_standard_file(self, file_path, file_type, permission):
        """PDF, Word ve PowerPoint için standart işleme."""
        loader = None
        try:
            if file_type == "pdf":
                loader = PyPDFLoader(file_path)
            elif file_type == "word":
                loader = Docx2txtLoader(file_path)
            elif file_type == "powerpoint":
                loader = UnstructuredPowerPointLoader(file_path)

            if not loader:
                return

            raw_docs = loader.load()
            chunks = self.text_splitter.split_documents(raw_docs)

            filename = os.path.basename(file_path)

            for i, chunk in enumerate(chunks):
                # Sayfa numarasını güvenli alma
                page = chunk.metadata.get("page", chunk.metadata.get("page_number", 1))
                if isinstance(page, int) and file_type == "pdf":
                     page += 1

                enriched_meta = {
                    "source": filename,
                    "page_number": page,
                    "chunk_number": len(self.processed_chunks) + 1,
                    "permission": permission,
                    "file_type": file_type,
                    "original_source": file_path
                }
                chunk.metadata = enriched_meta
                self.processed_chunks.append(chunk)

            print(f"  -> {file_type.upper()} işlendi: {len(chunks)} parça.")

        except Exception as e:
            print(f"  !! Hata ({file_type} - {os.path.basename(file_path)}): {e}")

    def save_data(self, output_path):
        with open(output_path, "wb") as f:
            pickle.dump(self.processed_chunks, f)
        print(f"\nToplam {len(self.processed_chunks)} chunk kaydedildi: {output_path}")


In [ ]:
def main():
    processor = DriveDocumentProcessor()

    print(f"Tarama Başlıyor: {BASE_PATH}")

    if not os.path.exists(BASE_PATH):
        print(f"HATA: Belirtilen yol bulunamadı! -> {BASE_PATH}")
        print("Lütfen Drive yolunuzu kontrol edin.")
        return

    # Klasörleri gez
    for folder in TARGET_FOLDERS:
        folder_path = os.path.join(BASE_PATH, folder)

        if not os.path.exists(folder_path):
            print(f"Uyarı: Klasör bulunamadı, geçiliyor -> {folder}")
            continue

        print(f"\n[{folder.upper()}] Klasörü Taranıyor...")

        files = os.listdir(folder_path)
        if not files:
            print("  (Klasör boş)")

        for filename in files:
            file_path = os.path.join(folder_path, filename)

            # Gizli dosyaları (.DS_Store vs) atla
            if filename.startswith("."):
                continue

            # Yetkiyi belirle
            permission = get_file_permission(filename, folder)

            # Dosya türüne göre yönlendir
            if folder == "excel" and (filename.endswith(".xlsx") or filename.endswith(".xls")):
                processor.process_excel(file_path, permission)

            elif folder == "pdf" and filename.endswith(".pdf"):
                processor.process_standard_file(file_path, "pdf", permission)

            elif folder == "word" and (filename.endswith(".docx") or filename.endswith(".doc")):
                processor.process_standard_file(file_path, "word", permission)

            elif folder == "powerpoint" and (filename.endswith(".pptx") or filename.endswith(".ppt")):
                processor.process_standard_file(file_path, "powerpoint", permission)

            else:

                pass

    # Çıktıyı Kaydet
    output_file = "/content/drive/MyDrive/1505 Chunking/tum_dokumanlar_chunked.pkl"

    if os.path.exists(BASE_PATH):

         output_file = os.path.join(os.path.dirname(BASE_PATH), "1505_chunk_output.pkl")

    processor.save_data(output_file)

if __name__ == "__main__":
    main()

Tarama Başlıyor: /content/drive/MyDrive/1505 Chunking/Doküman

[PDF] Klasörü Taranıyor...
  -> PDF işlendi: 67 parça.
  -> PDF işlendi: 53 parça.
  -> PDF işlendi: 24 parça.
  -> PDF işlendi: 103 parça.
  -> PDF işlendi: 13 parça.
  -> PDF işlendi: 51 parça.
  -> PDF işlendi: 44 parça.
  -> PDF işlendi: 21 parça.
  -> PDF işlendi: 27 parça.
  -> PDF işlendi: 22 parça.
  -> PDF işlendi: 5 parça.
  -> PDF işlendi: 7 parça.
  -> PDF işlendi: 35 parça.
  -> PDF işlendi: 7 parça.
  -> PDF işlendi: 14 parça.
  -> PDF işlendi: 5 parça.
  -> PDF işlendi: 15 parça.
  -> PDF işlendi: 7 parça.
  -> PDF işlendi: 5 parça.
  -> PDF işlendi: 3 parça.
  -> PDF işlendi: 3 parça.
  -> PDF işlendi: 4 parça.
  -> PDF işlendi: 2 parça.
  -> PDF işlendi: 2 parça.
  -> PDF işlendi: 2 parça.
  -> PDF işlendi: 2 parça.
  -> PDF işlendi: 3 parça.
  -> PDF işlendi: 4 parça.
  -> PDF işlendi: 6 parça.
  -> PDF işlendi: 26 parça.
  -> PDF işlendi: 5 parça.
  -> PDF işlendi: 4 parça.
  -> PDF işlendi: 9 parça.

[EX

*view*

In [1]:
import os
import sys
import pickle
from google.colab import drive


In [2]:
try:
    from langchain.docstore.document import Document
except ImportError:
    import subprocess
    print("Gerekli kütüphane (langchain) kuruluyor...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain", "langchain-community"])
    from langchain.docstore.document import Document


In [3]:
POSSIBLE_PATHS = [
    "/content/drive/MyDrive/1505 Chunking/processed_chunks_data.pkl",
    "/content/drive/MyDrive/1505 Chunking/1505_chunk_output.pkl",
    "/content/drive/MyDrive/1505 Chunking/tum_dokumanlar_chunked.pkl"
]

def find_file():
    for path in POSSIBLE_PATHS:
        if os.path.exists(path):
            return path
    return None

def preview_data(file_path):
    print(f"Dosya Yükleniyor: {file_path} ...")

    try:
        with open(file_path, "rb") as f:
            chunks = pickle.load(f)
    except Exception as e:
        print(f"HATA: Dosya okunamadı. {e}")
        return

    total_count = len(chunks)
    print(f"Başarıyla Yüklendi! Toplam Chunk Sayısı: {total_count}")
    print("="*60)

    print(f"\n>>> İlk 10 Chunk")
    for i, chunk in enumerate(chunks[:10]):
        source = chunk.metadata.get('source', 'Bilinmiyor')
        perm = chunk.metadata.get('permission', 'Yok')
        content_preview = chunk.page_content.replace('\n', ' ')[:100]

        print(f"[{i+1}] Dosya: {source} | Yetki: {perm}")
        print(f"    İçerik: {content_preview}...")
        print("-" * 30)

    print(f"\n>>> Dosya Türü Bazlı Örnekler")

    target_types = ["pdf", "excel", "word", "powerpoint"]
    # Türlere göre grupla
    samples = {t: [] for t in target_types}

    for chunk in chunks:
        ftype = chunk.metadata.get("file_type", "unknown")
        if ftype in samples and len(samples[ftype]) < 3:
            samples[ftype].append(chunk)

    for ftype in target_types:
        items = samples.get(ftype, [])
        print(f"\n--- TÜR: {ftype.upper()} ({len(items)} örnek gösteriliyor) ---")

        if not items:
            print("   (Bu dosya türüne ait veri bulunamadı)")
            continue

        for idx, item in enumerate(items):
            if ftype == 'excel':
                loc_label = "Satır"
                loc_val = item.metadata.get('row_number', '?')
            else:
                loc_label = "Sayfa"
                loc_val = item.metadata.get('page_number', '?')

            src = item.metadata.get('source')
            text = item.page_content.replace('\n', ' ')[:150]

            print(f"{idx+1}. {src} ({loc_label}: {loc_val})")
            print(f"   Veri: {text}...")


In [4]:
found_path = find_file()

if found_path:
    preview_data(found_path)
else:
    print("HATA: .pkl dosyası bulunamadı!")
    print("Lütfen aşağıdaki yolları kontrol edin veya dosya yolunu manuel olarak düzeltin:")
    for p in POSSIBLE_PATHS:
        print(f" - {p}")

Dosya Yükleniyor: /content/drive/MyDrive/1505 Chunking/1505_chunk_output.pkl ...
Başarıyla Yüklendi! Toplam Chunk Sayısı: 754

>>> İlk 10 Chunk
[1] Dosya: 207858.pdf | Yetki: user
    İçerik: Senato Karar Eki    BURSA ULUDAĞ ÜNİVERSİTESİ İŞ SAĞLIĞI VE GÜVENLİĞİ YÖNERGESİ  BİRİNCİ BÖLÜM  Amaç...
------------------------------
[2] Dosya: 207858.pdf | Yetki: user
    İçerik: Tanımlar  MADDE 4- (1) Bu Yönerge’de geçen,  a) Acil durum: İşyerinin tamamında veya bir kısmında me...
------------------------------
[3] Dosya: 207858.pdf | Yetki: user
    İçerik: d) Birim: Bir işyeri sosyal güvenlik sicil numarası ve/veya birim NACE kod numarasına  sahip olup İş...
------------------------------
[4] Dosya: 207858.pdf | Yetki: user
    İçerik: Senato Karar Eki    f) Çalışan: Çırak ve stajyerler dahil kendi özel kanunlarındaki statülerine bakı...
------------------------------
[5] Dosya: 207858.pdf | Yetki: user
    İçerik: sahip kişileri,  ı) Geçici iş ilişkisi: Eğitim, görevlendirme gibi nedenlerl